<a href="https://colab.research.google.com/github/3104305x/1_project_x/blob/main/todo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

处理图片服务器中图片文件, 删除无用图片, 重复图片仅保留一张, 删除其它多余图片

In [ ]:
# prompt: 使用pymysql与mysql数据库建立连接, 读取 数据库 www_xxx_com 中的 表 fa_cms_archives 中的 image 字段内容,  读取 数据表 fa_cms_addonnews 中 content 字段 内容

import pymysql

# 连接数据库
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='123456', db='www_xxx_com')

# 创建游标
cursor = conn.cursor()

# 读取数据库www_xxx_com中的表fa_cms_archives中的image字段内容
cursor.execute('SELECT image FROM fa_cms_archives')
images = cursor.fetchall()

# 读取数据表fa_cms_addonnews中的content字段内容
cursor.execute('SELECT content FROM fa_cms_addonnews')
contents = cursor.fetchall()

# 关闭游标和连接
cursor.close()
conn.close()


image 字段内容为相对路径 '/uploads/20220902/5154837b7940290575854283ced96b16.png'
content 字段内容为 html文本, 需要使用 BeautifulSoup 解析为html对象, 然后获取所有img对象的src内容

In [ ]:
# prompt: content 字段内容为 html文本, 需要使用 BeautifulSoup 解析为html对象, 然后获取所有img对象的src内容

from bs4 import BeautifulSoup
import re
image_set = set()
for content in contents:
  soup = BeautifulSoup(content[0], 'html.parser')
  imgs = soup.find_all('img')
  for img in imgs:
    src = img.get('src')
    if src:
      image_set.add(src)


In [ ]:
# prompt: 修改上述代码, 如果 src 地址是包含 域名 的url, 如 'https://www.jk5u.com/uploads/20220906/d255344f5fdcbca32ad1fb42a6bf1793.jpeg', 仅保留相对路径, 如 '/uploads/20220906/d255344f5fdcbca32ad1fb42a6bf1793.jpeg'

image_set = set()
for content in contents:
  soup = BeautifulSoup(content[0], 'html.parser')
  imgs = soup.find_all('img')
  for img in imgs:
    src = img.get('src')
    if src:
      if 'http' in src:
        src = src[src.find('/uploads/'):]
      image_set.add(src)


根据图片相对路径, 查找在 图片服务器 中的图片, 并对图片进行去重操作